In [1]:
### uncomment to display figures
%matplotlib inline

# Demonstrates benchmarking functionality - SASMAS Soil Moisture

#### Benchmarking dataset information
The soil moisture content (in mm) dataset consists of time-series of soil water content reflectometer measurements at various depths within the profile (top:0-5cm, shallow:0-30cm, middle:30-60cm, deep:60-90cm) within the Upper Hunter River
<br>
##### Source:
Rüdiger, C., Hancock, G., Hemakumara, H.M., Jacobs, B., Kalma, J.D., Martinez, C., Thyer, M., Walker, J.P., Wells, T. and Willgoose, G.R., 2007. Goulburn River experimental catchment data set. Water Resources Research, 43(10): W10403.

This notebook goes through the following steps:

1. Import required libraries
2. Set up benchmarking configuration <br>
 2.1 Catchments to be benchmarked<br>
 2.2 Define observation inputs<br>

3. Create benchmark object<br>
4. Add models to be benchmarked <br>
 4.1 Select or unselect models<br> 
5. View benchmarking statistics<br>
6. View benchmarking plots<br>
7. Statistics plotting<br>

### 1. Import required libraries

In [6]:
from awrams.benchmarking.benchmark import BenchmarkSoilMoisture
from awrams.utils import datetools as dt
import awrams.benchmarking.config as cfg
import awrams.benchmarking.meta.sasmas as sasmas

from awrams.models.settings import TRAINING_DATA_PATH

### 2.  Set up benchmarking configuration
Comparison against observed streamflow <br>
You can use your own data in csv form similar to the example provided. <br>
It just needs to have column names matching the names used in extracting AWRA data

#### 2.1 Sites to be benchmarked
Soil moisture comparisons at SASMAS sites

In [7]:
sasmas_data_path = TRAINING_DATA_PATH + '/benchmarking/sasmas/' #  # the sasmas data has been pre-processed into 5 files [top, shallow, middle, deep, profile]

site_list = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'M1', 'M2', 'M3', 
             'M4', 'M5', 'M6', 'M7', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7']

mod_site_list = ['SASMAS Soil moisture_' + site for site in site_list]

### 3.  Create the benchmark object:<br> 

An object of "Benchmark" class is created by defining what variable is to be benchmarked. 
Everything else gets progressively added,  and statistics are calculated when the observation and model outputs are added.

In [8]:
sas = BenchmarkSoilMoisture("SASMAS", "soil moisture", sasmas.meta)

# Specify benchmarking period
sas.period = dt.dates('2003-2011')

# Add observations and catchment subset [the id list needs to be present in the column names of the observation file]
sas.load(sasmas_data_path,mod_site_list,convert_units=100.)

In [9]:
sorted(sas.sites)

['G1',
 'G2',
 'G3',
 'G4',
 'G5',
 'G6',
 'K1',
 'K2',
 'K3',
 'K4',
 'K5',
 'K6',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'S1',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7']

In [10]:
cfg.MONTHLY_REJECTION_THRESHOLD = 15 # Minimum number of available obs days before monthly stats are calculated

In [11]:
cfg.SM_MODEL_VARNAMES

['s0_avg', 'ss_avg', 'sd_avg']

In [12]:
cfg.SM_MODEL_LAYERS

{'s0_avg': 100.0, 'sd_avg': 5000.0, 'ss_avg': 900.0}

### 4. Add models to be benchmarked
Any number of models can be simulataneously compared
This step processes the data and calculates all the statistics [can take a while]

In [ ]:
path = sasmas_data_path+"/awral_${v}.csv"
sas.add_model("AWRAMSI.v4_0.AWRAL", csv_path=path)

In [ ]:
path = sasmas_data_path+"/AWRAMSI_v5QES_AWRAL_SASMAS_${v}.csv" 
sas.add_model("AWRAMSI.v5_0.AWRAL", csv_path=path)

#### 4.1 Show list of loaded or selected models
list of loaded models is available with <tab> activated dropdown by typing "et.models."<br>
can "select" or "unselect" models for displaying

In [ ]:
sas.benchmark.top.selection

### 5. View benchmarking statistics
Summary percentiles can be printed out by specifying a statistic from: <br> 
"grand_f", "nse", "bias_relative", "pearsons_r" (default), "mean"  <br>
to the 'stat_percentiles' function<br>
The timeframe defaults to monthly, but can be specified

These tables are pandas dataframes, so they can be exported to csv

In [ ]:
sas.benchmark.deep.stat_percentiles('fobj',freq='m')

In [ ]:
sas.benchmark.deep.stat_percentiles('grand_f',freq='m')

In [ ]:
sas.benchmark.top.stat_percentiles('bias')

In [ ]:
sas.benchmark.shallow.stat_percentiles('nse','daily')

In [ ]:
sas.benchmark.shallow.data_percentiles()

### 6. View benchmarking plots [time series, Regression]

specify frequency by "freq=d" for daily, "freq=m" for monthly, "freq=y" for yearly<br>
can customise titles, labels, scaling etc using standard matplotlib keyword arguments

In [ ]:
sas.benchmark.top.selection

In [ ]:
p = sas.benchmark.shallow.plot_timeseries('G1','raw')

In [ ]:
p = sas.benchmark.top.plot_timeseries('G1')

In [ ]:
p = sas.benchmark.shallow.plot_timeseries('G1')

In [ ]:
p = sas.benchmark.middle.plot_timeseries('G1')

In [ ]:
p = sas.benchmark.deep.plot_timeseries('G1')

In [ ]:
p = sas.benchmark.profile.plot_timeseries('G1')

In [ ]:
p = sas.benchmark.shallow.plot_regression('M1',xlim=[-1,60],ylim=[-1,60])

### 7. Statistics plotting
specify statistic type from "fobj", "nse", "rmse", "bias_relative", "pearsons_r" (default), "mean" and <br> frequency from 'd', 'm', 'y'

In [ ]:
p = sas.benchmark.shallow.plot_box('pearsons_r','daily')

In [ ]:
p = sas.benchmark.shallow.plot_cdf('pearsons_r')

In [ ]:
p = sas.benchmark.shallow.plot_box('nse','daily',ylim=[-1,1])